This Python script demonstrates how to build a simple, interactive chatbot using the Azure AI Inference SDK. It's designed to be a clear and well-documented example, perfect for a GitHub repository. The core functionality revolves around managing a conversation's history by continuously updating a list of messages.

## Explanation of the Code
# 1. Setup and Initialization
import statements: The script begins by importing necessary classes from the azure-ai-inference package and other standard libraries.

ChatCompletionsClient: The main class for interacting with Azure's chat completion models.

SystemMessage, UserMessage, AssistantMessage: These classes represent the different roles in a conversation (system, user, and the AI assistant).

AzureKeyCredential: A class to securely handle your API key for authentication.

os and dotenv: Used for loading environment variables, which is the best practice for keeping sensitive information like API keys out of your source code.

Loading Environment Variables: The load_dotenv() function reads key-value pairs from a .env file in the same directory. This prevents hardcoding secrets. The script then retrieves the Azure endpoint, API key, and model name from these variables. An if statement checks that all variables were successfully loaded, exiting with an error message if not.

Client Initialization: The ChatCompletionsClient is instantiated. This object acts as the interface to the Azure AI service. You provide it with the endpoint URL, the credential (your API key), and the api_version to ensure compatibility.

# 2. Conversation Management
messages list: A list named messages is created to store the entire conversation history. It is initialized with a SystemMessage. This message sets the "persona" or overall behavior of the AI for the entire chat session (e.g., "You are a helpful assistant."). This is a crucial step for guiding the model's responses.

Interactive Loop (while True): The script enters an infinite loop, allowing the user to have a continuous conversation with the bot.

# 3. The Chatbot's Logic
User Input: The line user_input = input("You: ") prompts the user for their message.

Exit Condition: An if statement checks if the user typed 'quit'. If so, the loop breaks, and the program ends.

Appending User Message: The user's input is packaged into a UserMessage object, which is then appended to the messages list. This adds the user's latest query to the conversation history.

API Call: client.complete(messages=messages, ...) sends the entire messages list to the Azure AI model. The model receives all previous turns of the conversation (system message, user messages, and assistant responses) and uses this context to generate a new, relevant response.

Parsing and Printing Response:

response.choices[0].message.content: The model's response is a complex object. This line accesses the first (and in this case, only) completion choice and extracts the actual text content of the message.

The print statement displays the assistant's response to the user.

Appending Assistant Message: Just as with the user's message, the assistant's response is converted into an AssistantMessage object and appended to the messages list. This is the most important step for maintaining context. By adding the AI's response to the history, the next time the loop runs, the model will remember what it just said.

This process of adding both user and assistant messages to the messages list, and then sending the complete list with each new request, is the fundamental principle of building a stateful, multi-turn chatbot.

In [2]:
# Install the necessary package if you haven't already
# It's good practice to include this for anyone trying to run the code.
# !pip install azure-ai-inference python-dotenv

import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

# --- Step 1: Configuration and Setup ---

# Load environment variables from a .env file.
# This is a critical step for secure credential management, preventing API keys
# from being hardcoded directly into the script.
load_dotenv()

# Retrieve values from environment variables.
# os.getenv() is preferred over os.environ[] as it returns None if the
# variable is not found, avoiding a KeyError.
endpoint = os.getenv("AZURE_AI_INFERENCE_ENDPOINT")
api_key = os.getenv("AZURE_AI_INFERENCE_API_KEY")
model_name = os.getenv("AZURE_AI_INFERENCE_MODEL_NAME")

# Check if environment variables are loaded correctly (optional but good practice)
if not all([endpoint, api_key, model_name]):
    print("Error: Please ensure AZURE_AI_INFERENCE_ENDPOINT, AZURE_AI_INFERENCE_API_KEY, and AZURE_AI_INFERENCE_MODEL_NAME are set in your .env file.")
    exit()

# Initialize the ChatCompletionsClient.
# This client is the primary interface for sending chat completion requests to Azure AI.
client = ChatCompletionsClient(
    endpoint=endpoint,                          # The URL of your Azure AI service endpoint.
    credential=AzureKeyCredential(api_key),     # Your API key for authentication.
    api_version="2024-05-01-preview"            # Specifies the API version for compatibility.
)

# --- Step 2: Conversation Initialization ---

# Start with a system message to set the bot's overall behavior or persona.
# This list will store the entire conversation history.
messages = [
    SystemMessage(content="You are a helpful assistant.")
]

print("--- Chatbot is ready! ---")
print("Type your message and press Enter. Type 'quit' to exit.")
print("-------------------------")

# --- Step 3: Main Chat Loop ---

# This loop allows for a continuous, multi-turn conversation.
while True:
    # Get user input from the console.
    user_input = input("Ask Question: ")

    # Check for the exit command.
    if user_input.lower() == 'quit':
        print("Goodbye! Thanks for chatting.")
        break

    # Add the user's message to the conversation history.
    # The 'messages' list is the key to maintaining context.
    messages.append(UserMessage(content=user_input))

    # --- Step 4: Make the API Call with Conversation History ---

    # Make a completion call to the Azure AI model.
    # The 'messages' parameter now contains the entire chat history.
    try:
        response = client.complete(
            messages=messages,
            max_tokens=2048,  # Limits the maximum length of the response.
            model=model_name  # Specifies which deployed model to use.
        )

        # Extract the content of the model's response.
        assistant_response_content = response.choices[0].message.content

        # Print the AI's response to the user.
        print(f"Assistant: {assistant_response_content}")

        # --- Step 5: Update Conversation History with Assistant's Response ---

        # Add the assistant's response to the conversation history.
        # This is crucial for the model to "remember" what it just said
        # in the next turn of the conversation.
        messages.append(AssistantMessage(content=assistant_response_content))

    except Exception as e:
        print(f"An error occurred: {e}")
        # Optionally, remove the last user message to avoid a broken conversation state.
        messages.pop()
        break # Exit the loop on error.

--- Chatbot is ready! ---
Type your message and press Enter. Type 'quit' to exit.
-------------------------
Assistant: <think>
Okay, the user just said "hi". A simple greeting, probably testing the waters or starting a casual interaction. 

Hmm, they might be new here or just want to check if I'm responsive. No context given yet, so best to keep it warm and open-ended. 

I'll mirror their tone—friendly but not overbearing. The smiley emoji feels appropriate since it's a low-stakes opener. 

Key goals: 
- Acknowledge the greeting 
- Encourage further conversation 
- Leave the ball in their court 

No need to dive into solutions yet. Just set a welcoming vibe. "How can I help?" keeps it versatile—they can ask a question, rant, or just chat. 

...And sent. Let's see where they take it.
</think>
Hello! 😊 How can I help you today?
Assistant: <think>
Okay, the user just asked about stars after a simple greeting. They might be curious about astronomy, doing homework, or just fascinated by the